In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
daily23 = pd.read_csv("2021-2023\CSB_2021-2023_Daily_Weather.csv")
hourly23 = pd.read_csv("2021-2023\CSB_2021-2023_Hourly_Weather.csv")
#all_2023 = pd.read_csv("CSB_2021-2023_All.csv")

In [4]:
dd = daily23.copy()
hd = hourly23.copy()
#all2023 = all_2023.copy()

In [25]:
hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         21864 non-null  datetime64[ns]
 1   Hour         21864 non-null  int64         
 2   Casual       21864 non-null  float64       
 3   Member       21864 non-null  float64       
 4   Total_rides  21864 non-null  float64       
 5   temp(°C)     21864 non-null  float64       
 6   humidity(%)  21864 non-null  int64         
 7   WMO code     21864 non-null  int64         
 8   windspeed    21864 non-null  float64       
 9   season       21864 non-null  int64         
 10  Season       21864 non-null  object        
 11  yr           21864 non-null  object        
 12  mnth         21864 non-null  int32         
 13  holiday      21864 non-null  bool          
 14  weekday      21864 non-null  int32         
 15  workingday   21864 non-null  bool          
 16  Week

In [6]:
#rename
dd = dd.rename(columns={"temperature_2m_mean (°C)": "temp(°C)", "relativehumidity_2m (%)": "humidity(%)", "weathercode (wmo code)": "WMO code", 'apparent_temperature_mean (°C)': "atemp(°C)", 'windspeed_10m_max (km/h)': "windspeed"})
hd = hd.rename(columns={"temperature_2m (°C)": "temp(°C)", "relativehumidity_2m (%)": "humidity(%)", "weathercode (wmo code)": "WMO code", 'windspeed_10m (km/h)': "windspeed"})

In [7]:
dd["Date"] = pd.to_datetime(dd['Date'])
hd["Date"] = pd.to_datetime(hd['Date'])

hd[['Casual', 'Member', 'Total_rides']] = hd[['Casual', 'Member', 'Total_rides']].fillna(0) #fill NaN

In [26]:
display(dd.isna().sum())
display(hd.isna().sum())

Date           0
Casual         0
Member         0
Total_rides    0
WMO code       0
temp(°C)       0
atemp(°C)      0
windspeed      0
season         0
Season         0
yr             0
mnth           0
holiday        0
weekday        0
workingday     0
Weekday        0
weathersit     0
dtype: int64

Date           0
Hour           0
Casual         0
Member         0
Total_rides    0
temp(°C)       0
humidity(%)    0
WMO code       0
windspeed      0
season         0
Season         0
yr             0
mnth           0
holiday        0
weekday        0
workingday     0
Weekday        0
weathersit     0
dtype: int64

In [9]:
from datetime import datetime
def season(Date):
    day_of_year = Date.timetuple().tm_yday
    if 80 <= day_of_year <= 171:return 2  # Spring
    elif 172 <= day_of_year <= 263: return 3  # Summer
    elif 264 <= day_of_year <= 355: return 4  # Autumn
    else: return 1  # Winter
dd['season'] = dd['Date'].apply(season)
hd["season"] = hd['Date'].apply(season)

def season_name(Date):
    day_of_year = Date.timetuple().tm_yday
    if 80 <= day_of_year <= 171:return 'Spring'
    elif 172 <= day_of_year <= 263: return 'Summer'
    elif 264 <= day_of_year <= 355: return 'Autumn'
    else: return 'Winter'
dd['Season'] = dd['Date'].apply(season_name)
hd["Season"] = hd['Date'].apply(season_name)

In [10]:
hd['humidity(%)'].unique()

array([ 90,  88,  87,  81,  77,  75,  72,  71,  70,  69,  80,  92,  94,
        95,  96,  97,  98,  99,  93,  66,  62,  63,  64,  76,  85,  89,
        91,  86,  82,  74,  61,  59,  60,  65,  83,  73,  79,  68,  57,
        52,  51,  50,  54,  56,  67,  49,  55,  78,  84,  58,  48,  43,
        39,  36,  34,  33,  38,  41,  45,  47,  53,  46,  44,  37,  42,
        40,  35,  32, 100,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  20,  19], dtype=int64)

In [27]:
unique_humidity = hd[hd['season'] ==3]['humidity(%)'].unique()
print(unique_humidity)

[ 78  81  85  94  95  96  97  89  83  75  67  61  58  54  51  70  76  77
  92  93  91  74  63  57  60  68  71  82  65  55  46  43  41  40  39  38
  37  42  53  73  87  88  52  48  45  44  47  56  59  69  79  90  50  49
  72  80  86  84  66  62  64  35  34  36  98  99  31  30 100]


In [12]:
# def season(Date):
#     month= Date.month
#     if 3 <= month <= 5 : return 2 #spring
#     if 6 <= month <= 8: return 3 #summer
#     if 9 <= month <= 11: return 4 #autumn
#     else: return 1 #winter
# def year(Date):
#     year = Date.year
#     if year == '2022': return 0
#     else: return 1
    
#hd["season"] = hd['Date'].apply(season)
#white_wine['quality_category'] = white_wine['quality'].apply()

In [13]:
#year: 2 for 2021, 3 for 2022, 4 for 2023 (2011 is 0 and 2012 is 1)
dd['yr'] = dd['Date'].apply(lambda value: '2' if value.year == 2021 else ('3' if value.year == 2022 else '4'))
hd['yr'] = hd['Date'].apply(lambda value: '2' if value.year == 2021 else ('3' if value.year == 2022 else '4'))
#month
dd['mnth'] = dd['Date'].dt.month
hd['mnth'] = hd['Date'].dt.month

In [14]:
holidays = [
    "01.01.2021", "18.01.2021", "20.01.2021", "15.02.2021", "16.04.2021", 
    "31.05.2021", "18.06.2021", "05.07.2021", "06.09.2021", "11.10.2021",
    "11.11.2021", "25.11.2021", "24.12.2021", "01.01.2022", "17.01.2022", 
    "21.02.2022", "15.04.2022", "30.05.2022", "19.06.2022", "20.06.2022", 
    "04.07.2022", "05.09.2022", "10.10.2022", "11.11.2022", "24.11.2022", 
    "22.12.2022", "26.12.2022", "01.01.2023", "02.01.2023", "16.01.2023", 
    "20.02.2023", "17.04.2023", "29.05.2023", "19.06.2023", "04.07.2023", 
    "04.09.2023", "09.10.2023", "10.11.2023", "23.11.2023", "25.12.2023"] #source https://edpm.dc.gov/issuances/legal-public-holidays-2023/

holidays = [pd.to_datetime(date, format="%d.%m.%Y") for date in holidays]

dd['holiday'] = dd['Date'].isin(holidays).astype(int)
hd['holiday'] = hd['Date'].isin(holidays).astype(int)

In [15]:
dd['weekday'] = dd['Date'].dt.weekday
hd['weekday'] = hd['Date'].dt.weekday

#check if its a holiday and weekend and chang holiday to boolean
dd["holiday"] = dd["holiday"].astype("bool")
hd["holiday"] = hd["holiday"].astype("bool")
dd['workingday'] = np.where((dd['weekday'] > 4) | dd['holiday'], False, True)
hd['workingday'] = np.where((hd['weekday'] > 4) | hd['holiday'], False, True)

In [16]:
hd['Weekday'] = hd['Date'].dt.day_name()
dd['Weekday'] = dd['Date'].dt.day_name()

In [17]:
hd

,Date,Hour,Casual,Member,Total_rides,temp(°C),humidity(%),WMO code,windspeed,season,Season,yr,mnth,holiday,weekday,workingday,Weekday
0,2021-01-01,0,20.0,26.0,46.0,2.4,90,1,10.2,1,Winter,2,1,True,4,False,Friday
1,2021-01-01,1,23.0,23.0,46.0,1.8,88,1,10.0,1,Winter,2,1,True,4,False,Friday
2,2021-01-01,2,20.0,28.0,48.0,1.1,87,2,10.1,1,Winter,2,1,True,4,False,Friday
3,2021-01-01,3,9.0,7.0,16.0,1.6,81,2,10.6,1,Winter,2,1,True,4,False,Friday
4,2021-01-01,4,7.0,5.0,12.0,1.6,77,2,9.9,1,Winter,2,1,True,4,False,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21859,2021-02-19,3,0.0,0.0,0.0,-1.6,93,51,10.4,1,Winter,2,2,False,4,True,Friday
21860,2021-03-14,2,0.0,0.0,0.0,2.9,56,0,13.9,1,Winter,2,3,False,6,False,Sunday
21861,2022-01-04,2,0.0,0.0,0.0,-8.7,77,0,10.1,1,Winter,3,1,False,1,True,Tuesday
21862,2022-03-13,2,0.0,0.0,0.0,-5.1,50,0,21.1,1,Winter,3,3,False,6,False,Sunday


In [18]:
#create weather situation
hd['weathersit'] = hd['WMO code'].apply(lambda value: '1' if value < 20 else ('2' if value <30 else ('3' if value < 40 else '4')))
dd['weathersit'] = dd['WMO code'].apply(lambda value: '1' if value < 20 else ('2' if value <30 else ('3' if value < 40 else '4')))

In [28]:
hd.head()

,Date,Hour,Casual,Member,Total_rides,temp(°C),humidity(%),WMO code,windspeed,season,Season,yr,mnth,holiday,weekday,workingday,Weekday,weathersit
0,2021-01-01,0,20.0,26.0,46.0,2.4,90,1,10.2,1,Winter,2,1,True,4,False,Friday,1
1,2021-01-01,1,23.0,23.0,46.0,1.8,88,1,10.0,1,Winter,2,1,True,4,False,Friday,1
2,2021-01-01,2,20.0,28.0,48.0,1.1,87,2,10.1,1,Winter,2,1,True,4,False,Friday,1
3,2021-01-01,3,9.0,7.0,16.0,1.6,81,2,10.6,1,Winter,2,1,True,4,False,Friday,1
4,2021-01-01,4,7.0,5.0,12.0,1.6,77,2,9.9,1,Winter,2,1,True,4,False,Friday,1


In [29]:
dd

,Date,Casual,Member,Total_rides,WMO code,temp(°C),atemp(°C),windspeed,season,Season,yr,mnth,holiday,weekday,workingday,Weekday,weathersit
0,2021-01-01,371,590,961,63,2.3,-0.9,10.6,1,Winter,2,1,True,4,False,Friday,4
1,2021-01-02,2874,2844,5718,61,7.6,4.8,19.1,1,Winter,2,1,False,5,False,Saturday,4
2,2021-01-03,495,1133,1628,63,3.9,0.9,13.0,1,Winter,2,1,False,6,False,Sunday,4
3,2021-01-04,907,2074,2981,3,4.1,1.0,13.3,1,Winter,2,1,False,0,True,Monday,1
4,2021-01-05,1057,2004,3061,51,4.0,1.4,9.2,1,Winter,2,1,False,1,True,Tuesday,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,2023-06-26,4602,8373,12975,63,25.3,28.8,10.7,3,Summer,4,6,False,0,True,Monday,4
907,2023-06-27,4305,8542,12847,63,23.7,26.9,14.4,3,Summer,4,6,False,1,True,Tuesday,4
908,2023-06-28,5691,10150,15841,3,22.8,23.6,16.0,3,Summer,4,6,False,2,True,Wednesday,1
909,2023-06-29,5009,8807,13816,1,23.6,25.2,9.2,3,Summer,4,6,False,3,True,Thursday,1


In [30]:
hd.to_csv('hd_2021-23.csv', index=False)
dd.to_csv('dd_2021-23.csv', index=False)

In [22]:
# analyse events
#analyse weather
#maybe special weather for 
#how to calculate or get a atemp for all database